### Packages

In [20]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
import logging
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, Polygon, box, LineString
from shapely import ops
import folium
ox.config(use_cache=True, log_console=True)
print('OSM Version : {}'.format(ox.__version__))

%matplotlib widget

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
warnings.filterwarnings('ignore')

OSM Version : 1.1.2


### Configure logger

In [2]:
logging.basicConfig(level = logging.CRITICAL, format = "%(asctime)s: %(message)s")
logger = logging.getLogger('WMATA')
logger.setLevel(logging.INFO)
logger.info('WMATA Route Shape Matching.')

2022-01-05 10:38:53,813: WMATA Route Shape Matching.


### Directory and parameters

In [3]:
# Directory
data_dir = os.path.join('../data')
CRS = 'EPSG:4326'
CRS_METER = 'EPSG:3857'

In [4]:
# File names
gis_lines_geojson = os.path.join(data_dir,
                                 'gis_bsrt_line',
                                 'gis_bsrt_line.geojson')

gtfs_shapes_text = os.path.join(data_dir,
                                'gtfs_20210903',
                                'shapes.txt')

### Selection parameter

In [6]:
route_pattern_id = 'G801'
query_month = 20210903

### GIS bus route line

In [70]:
def read_all_gis_route_lines(gis_lines_geojson,
                             crs):
    """
    Get all GIS route line geometries in the data.

    :param gis_lines_geojson: Directory of GIS bus route geojson file including file name and extension.
    :param crs: Coordinate Reference System in "EPSG:XXXX" format.
    :return: A GeoDataFrame with line geometries for all available route patterns in specified CRS.
    """

    # Import WMATA GIS bus route line geometries
    gis_routes = gpd.read_file(gis_lines_geojson)
    gis_routes = gis_routes.to_crs(crs)
    gis_routes.columns = [c.lower() if c != 'geometry' else c for c in gis_routes.columns]

    # Create "shape_id" column
    gis_routes['pattern_id'] = gis_routes['gis_routec'].str.split('_').str[-1]
    gis_routes['shape_id'] = gis_routes['route'] + gis_routes['pattern_id']
    gis_routes = gis_routes.sort_values(by='shape_id')

    # Create start and end date in YYYYMM format for schedule based selection
    gis_routes['start_date'] = gis_routes['str_date'].astype(str).str[:10].str.replace('-', '').astype(int)
    gis_routes['end_date'] = gis_routes['end_date'].astype(str).str[:10].str.replace('-', '').astype(int)
    logger.info(f'Route line geometries returned for {gis_routes.shape_id.nunique()} unique route patterns.')

    return gis_routes.sort_values(by=['start_date'])


def read_single_gis_route_line(gis_lines_geojson,
                               crs,
                               route_pattern_id,
                               query_month):
    """
    Get single GIS line geometry.

    :param gis_lines_geojson: Directory of GIS bus route geojson file including file name and extension.
    :param crs: Coordinate Reference System in "EPSG:XXXX" format.
    :param route_pattern_id: Single route pattern ID in RRDD format (RR = Route Number, DD = Direction).
    :param query_month: Query date in YYYYMMDD format - must be in integer.
    :return: Relevant rows from the GIS bus route geojson with line geometries in specified CRS.
    """

    # Import WMATA GIS bus route line geometries
    gis_routes = read_all_gis_route_lines(gis_lines_geojson, crs)

    single_route = gis_routes[(gis_routes['shape_id'] == route_pattern_id) &
                              (gis_routes['start_date'] <= query_month) &
                              (gis_routes['end_date'] > query_month)].sort_values(by=['start_date'])
    logger.info(f'Route line geometries returned for route pattern {route_pattern_id}.')
    return single_route



In [8]:
# Extract select route pattern
active_gis_routes = read_single_gis_route_line(gis_lines_geojson, CRS, route_pattern_id, query_month)

2022-01-05 10:49:03,847: Route line geometries returned for 1147 unique route patterns.
2022-01-05 10:49:03,883: Route line geometries returned for route pattern G801.


### GTFS shapes

In [71]:
def read_gtfs_shapes(gtfs_shapes_text, crs, route_pattern_id):
    shapes = pd.read_csv(gtfs_shapes_text)
    shapes.columns = [c.lower() for c in shapes.columns]
    shapes['shape_id'] = shapes['shape_id'].str.replace(':', '')
    shapes = shapes[shapes.shape_id == route_pattern_id]
    shapes = create_point_gdf(shapes, 'shape_pt_lon', 'shape_pt_lat', crs)
    logger.info(f'Route shape geometries returned for route pattern {route_pattern_id} with {len(shapes)} points.')
    return shapes

In [72]:
shapes = read_gtfs_shapes(gtfs_shapes_text, CRS, route_pattern_id)

2022-01-05 11:25:00,534: Route shape geometries returned for route pattern G801 with 879 points.


### Line geometry from shapes

In [36]:
def get_line_geom_from_shapes(shapes_gdf, crs):
    shape_line_geometry = (shapes_gdf.sort_values(by = ['shape_id', 'shape_pt_sequence'])
                                 .groupby(['shape_id'])['geometry']
                                 .apply(lambda x: LineString(x.tolist())))
    logger.info(f'Route line geometries returned for route pattern {route_pattern_id} using {len(shapes)} points.')
    return gpd.GeoDataFrame(shape_line_geometry, 
                            geometry = 'geometry',
                            crs = CRS).reset_index()

In [37]:
shapes_line = get_line_geom_from_shapes(shapes_gdf = shapes, crs = CRS)

2022-01-05 11:05:37,522: Route line geometries returned for route pattern G801 using 879 points.


### Route buffer

In [88]:
def create_buffer(gdf, buffer_in_meter, crs, sort_columns = None):
    """
    Creates buffered polygons based in given value in meter.
    
    Args:
    -----
    gdf             : A GeoDataFrame.
    sort_columns    : A list column to sort the records
                      before creating buffer.
    buffer_in_meter : Distance in meter.
    
    Return:
    -------
    A GeoDataFrame with buffered polygons.    
    """
    # Sort records if column names are given
    if sort_columns is not None:
        gdf = gdf.copy().sort_values(by = sort_columns)
    else:
        gdf = gdf.copy()
    
    # Convert CRS to use meter
    gdf = gdf.to_crs('EPSG:3857')
    gdf['geometry'] = gdf.geometry.buffer(buffer_in_meter)
    return gdf.to_crs(CRS)

def create_union_of_buffer_geom(buff1, buff2, only_geom = False):
    if only_geom:
        return gpd.overlay(buff1, buff2, how = 'union').dissolve().geometry[0]
    else:
        buff = gpd.overlay(buff1, buff2, how = 'union').dissolve()
        return buff.geometry[0], buff

In [89]:
# Create buffers for GTFS lines and GIS lines
shapes_line_buffer = create_buffer(shapes_line, buffer_in_meter = 25, crs = CRS, sort_columns = ['shape_id'])
active_gis_routes_buffer = create_buffer(active_gis_routes, buffer_in_meter = 25, crs = CRS, sort_columns = ['shape_id'])[['shape_id', 'geometry']]

In [90]:
buffer_geom, route_buffer = create_union_of_buffer_geom(shapes_line_buffer, active_gis_routes_buffer)

### Export interim files

In [94]:
# Export shapes
shapes.to_file(os.path.join(data_dir, 
                            'interim',
                            f'{route_pattern_id}_ROUTE_SHAPES.geojson'))

# Export shape line
shapes_line.to_file(os.path.join(data_dir, 
                                 'interim',
                                 f'{route_pattern_id}_ROUTE_SHAPE_LINE.geojson'))

# Export GIS line
active_gis_routes.to_file(os.path.join(data_dir, 
                                       'interim',
                                       f'{route_pattern_id}_GIS_ROUTE_LINE.geojson'))

# Export buffer
route_buffer.to_file(os.path.join(data_dir, 
                                        'interim',
                                        f'{route_pattern_id}_ROUTE_BUFFER.geojson'))

### Import OSM

In [75]:
def get_graph_nodes_edges_sindex(graph_obj, spatial_index=True):
    """
    Create nodes and edges from graph with spatial indexes if specified.
    :param graph_obj: A graph object of street network.
    :param spatial_index: Boolean indicator to specify if spatial indexes should be returned as well.
    :return: Nodes and edges GeoDataFrame with nodes and edges spatial indexes if specified.
    """
    nodes, edges = ox.graph_to_gdfs(graph_obj, edges=True)
    if spatial_index:
        return nodes.reset_index(), edges.reset_index(), get_spatial_index(nodes), get_spatial_index(edges)
    else:
        return nodes.reset_index(), edges.reset_index()

In [9]:
G = ox.load_graphml(os.path.join(data_dir, 'osm_graph', 'wmata_osm_graph.graphml'))

In [76]:
# Nodes and edges from graph
nodes, edges, node_sindex, edge_sindex = get_graph_nodes_edges_sindex(G)

### Get list of nodes inside the buffer using edges

In [95]:
def get_node_list_using_edges_within_buffer(edges, edge_sindex, buffer_geom):
    """
    Returns list of nodes inside the buffer.
    """
 
    # Candidate edges
    edge_candidate_idx = list(edge_sindex.intersection(buffer_geom.bounds))
    candidate_edges = edges.iloc[edge_candidate_idx]
    intersecting_edges = candidate_edges[candidate_edges.intersects(buffer_geom)]
    u_list = intersecting_edges.loc[:,'u'].unique().tolist()
    v_list = intersecting_edges.loc[:,'v'].unique().tolist()
    node_list = list(set(u_list + v_list))
   
    logger.info(f'{len(intersecting_edges)} edges (of {len(edges)} total edges) found inside the buffer.')
    logger.info(f'{len(node_list)} nodes returned.')
    return node_list

In [96]:
node_list_from_edges = get_node_list_using_edges_within_buffer(edges, edge_sindex, buffer_geom)

2022-01-05 11:44:21,215: 816 edges (of 680826 total edges) found inside the buffer.
2022-01-05 11:44:21,218: 439 nodes returned.


### Get a subgraph using the list of nodes

In [97]:
def get_subgraph_using_node_list(G, node_list):
    """
    Returns subgraph from node list.
    """
    return G.subgraph(node_list).copy()

In [101]:
def convert_list_to_string(df, col):
    """
    Convert list-type cell value to string-type cell value.

    :param df: A DataFrame or GeoDataFrame.
    :param col: A list of columns with list-type cell values.
    :return: A DataFrame or GeoDataFrame with list-type cell values converted to string-type cell values.
    """
    return df.loc[:, col].apply(lambda x: str(x) if type(x) != list else ", ".join([str(item) for item in x]))


def clean_edge_fields_for_export(edge_df):
    """
    Convert list-type cell values to string-type cell values in edge fields for export.

    :param edge_df: An edge DataFrame or GeoDataFrame.
    :return: Clean DataFrame or GeoDataFrame for export.
    """
    col_list = ['osmid', 'name', 'highway', 'length', 'oneway']
    for col in col_list:
        edge_df.loc[:, f'{col}'] = convert_list_to_string(edge_df, col)
    return edge_df[['u', 'v'] + col_list + ['geometry']].copy()

In [102]:
#logging.info('Create subset graph network')
G_sub = get_subgraph_using_node_list(G, node_list_from_edges)

# Nodes and graphs from sub-graph
G_sub_nodes, G_sub_edges, G_sub_node_sindex, G_sub_edge_sindex = get_graph_nodes_edges_sindex(G_sub)

In [103]:
# Export subgraph
clean_edge_fields_for_export(G_sub_edges).to_file(os.path.join(data_dir, 
                                        'interim', f'{route_pattern_id}_SUBGRAPH_EDGES.geojson'))

### Clip the streets based on route buffer

In [104]:
# Clip the streets based on route buffer
G_sub_edges_clipped = gpd.clip(G_sub_edges, buffer_geom).copy()

# Export clipped subgraph
clean_edge_fields_for_export(G_sub_edges_clipped).to_file(os.path.join(data_dir, 
                                                                       'interim', f'{route_pattern_id}_SUBGRAPH_EDGES_CLIPPED.geojson'))

### Attach original length to clipped edges

In [108]:
# Subgraph in meter
G_sub_edges_meter = G_sub_edges.to_crs(CRS_METER)
G_sub_edges_meter['orig_length'] = G_sub_edges_meter.geometry.length

In [109]:
# Convert crs to use meter
G_sub_edges_clipped_meter = G_sub_edges_clipped.to_crs(CRS_METER)

# Merge original length before clipping
G_sub_edges_clipped_meter = (G_sub_edges_clipped_meter.merge(G_sub_edges_meter[['u', 'v', 'osmid', 'orig_length']], 
                                                             on = ['u', 'v', 'osmid',], 
                                                             how = 'left'))

### New columns to compare lengths

In [111]:
G_sub_edges_clipped_meter.loc[:, 'new_length'] = G_sub_edges_clipped_meter.geometry.length
G_sub_edges_clipped_meter.loc[:, 'compare_length_value'] = (G_sub_edges_clipped_meter['orig_length'].astype(float) * 0.01)
G_sub_edges_clipped_meter.loc[:,'new_length'].fillna(1, inplace = True)

condition = ((abs(G_sub_edges_clipped_meter.new_length - G_sub_edges_clipped_meter['orig_length'].astype(float))) 
             < G_sub_edges_clipped_meter.compare_length_value)
G_sub_edges_clipped_meter.loc[:,'org_vs_new_length'] = np.where(condition, 1, 1500)

### Hardclip edges

In [112]:
# Hardclip
G_sub_edges_hardclipped = G_sub_edges_clipped_meter[G_sub_edges_clipped_meter['org_vs_new_length'] == 1]

G_sub_edges_hardclipped['edge_key'] = tuple(zip(G_sub_edges_hardclipped['u'],
                                                G_sub_edges_hardclipped['v'],
                                                G_sub_edges_hardclipped['key']))

G_sub_hardclipped_edgelist = G_sub_edges_hardclipped['edge_key'].tolist()

# Right now, manual edit is not implemented
# as outlined in prevoius wmata code.

### Get the nearest edges of the start and end points

In [114]:
# Get first and last edges
first_pt_geom = shapes.reset_index(drop = True).iloc[0].geometry
last_pt_geom = shapes.reset_index(drop = True).iloc[-1].geometry

In [115]:
# Using the first and last stop geometry,
# find the nearest edges.
first_pt_edge = ox.get_nearest_edge(G_sub, 
                     (first_pt_geom.y, first_pt_geom.x),
                     return_geom = False,
                     return_dist = False)

last_pt_edge = ox.get_nearest_edge(G_sub, 
                    (last_pt_geom.y, last_pt_geom.x),
                    return_geom = False,
                    return_dist = False)

first_pt_edge_opposite = (first_pt_edge[1], first_pt_edge[0], first_pt_edge[2])
last_pt_edge_opposite = (last_pt_edge[1], last_pt_edge[0], last_pt_edge[2])
first_last_edgelist = [first_pt_edge,
                       first_pt_edge_opposite,
                       last_pt_edge,
                       last_pt_edge_opposite]

G_sub_hardclipped_first_last = G_sub_hardclipped_edgelist + first_last_edgelist

### Get the final hardclipped graph and edges

In [116]:
# Get the largest connected graph
G_sub_hardclipped = ox.utils_graph.get_largest_component(nx.edge_subgraph(G, G_sub_hardclipped_first_last))

In [117]:
# Export hardclipped nodes and edges
G_sub_hardclipped_nodes, G_sub_hardclipped_edges, G_sub_hardclipped_node_sindex, G_sub_hardclipped_sindex = get_graph_nodes_edges_sindex(G_sub_hardclipped)

In [122]:
# Export hardclipped edges
clean_edge_fields_for_export(G_sub_hardclipped_edges).to_file(os.path.join(data_dir, 
                                                                       'interim', f'{route_pattern_id}_SUBGRAPH_EDGES_HARDCLIPPED.geojson'))

# Export hardclipped nodes
G_sub_hardclipped_nodes.to_file(os.path.join(data_dir, 'interim', f'{route_pattern_id}_SUBGRAPH_NODES_HARDCLIPPED.geojson'))

# Export hardclipped nodes
ox.save_graphml(G_sub_hardclipped, os.path.join(data_dir, 'interim', f'{route_pattern_id}_SUBGRAPH_HARDCLIPPED.graphml'))